# Comparing the three algorithms by Neal

In [ ]:
import numpy as np
import scipy.stats as stats
import subprocess
import matplotlib.pyplot as plt
from google.protobuf.internal.decoder import _DecodeVarint32
import sys
sys.path.insert(0, '..')
from proto.py.algorithm_state_pb2 import AlgorithmState
import arviz as az
# import pip
# pip.main(["install", "arviz"])

# Utility to save files with Unix-like newlines
def save_np(filename, npobj):
    with open(filename, 'wb') as f:
        np.savetxt(f, npobj, fmt='%1.5f')

In [ ]:
# Generate data
rng = 20201124
np.random.seed(rng)
n = 200
mean1 = -3.0
mean2 = +3.0
norm1 = np.random.normal(loc=mean1, scale=1.0, size=int(n/2))
norm2 = np.random.normal(loc=mean2, scale=1.0, size=int(n/2))
uni_data = np.concatenate((norm1, norm2))
# Generate grid
uni_grid = np.arange(-10, +10, 0.1)

In [ ]:
# Save to file
save_np("../resources/csv/in/uni_data.csv", uni_data)
save_np("../resources/csv/in/uni_grid.csv", uni_grid)

In [ ]:
# True density of data
true_pdf = 0.5 * stats.norm.pdf(uni_grid, mean1, 1.0) + \
           0.5 * stats.norm.pdf(uni_grid, mean2, 1.0)

In [ ]:
# Define list of algorithms
algos = ["Neal2", "Neal3", "Neal8"]

In [ ]:
# Run the executable
for algo in algos:
    cmd = ["../build/run",
        "../algo_marg_settings.asciipb",
        "NNIG", "../resources/asciipb/nnig_ngg_prior.asciipb",
        "DP",   "../resources/asciipb/dp_gamma_prior.asciipb",
        f"../{algo}.recordio",
        "../resources/csv/in/uni_data.csv",
        "../resources/csv/in/uni_grid.csv",
        f"../resources/csv/out/uni_{algo}_dens.csv",
        f"../resources/csv/out/uni_{algo}_mass.csv",
        f"../resources/csv/out/uni_{algo}_nclu.csv",
        f"../resources/csv/out/uni_{algo}_clus.csv"
    ]
    output = subprocess.run(cmd, capture_output=True)
    print(output)

## Clustering

In [ ]:
# Read clusterings
clusterings = dict.fromkeys(algos)
for algo in algos:
    clusterings[algo] = np.loadtxt(f"../resources/csv/out/uni_{algo}_clus.csv")

In [ ]:
# Compare clusterings by counting misclassified points
print(np.linalg.norm(clusterings["Neal2"]-clusterings["Neal3"], 1))
print(np.linalg.norm(clusterings["Neal2"]-clusterings["Neal8"], 1))
print(np.linalg.norm(clusterings["Neal3"]-clusterings["Neal8"], 1))

## Density estimation

In [ ]:
# Densities
plt.figure(figsize=(16, 8))
for algo in algos:
    matr = np.genfromtxt(f"../resources/csv/out/uni_{algo}_dens.csv", delimiter=',')
    plt.plot(uni_grid, np.exp(np.mean(matr, axis=0)))
plt.plot(uni_grid, true_pdf, color="red", linestyle="--")
plt.legend(algos + ["true"])
plt.title("Univariate densities")

## Effective Sample Size

In [ ]:
# Utility to read file collector, courtesy of
# github.com/mberaha/utils/blob/master/proto_utils/py/recordio.py
def readManyFromFile(filename, msgType):
    out = []
    with open(filename, "rb") as fp:
        buf = fp.read()
    n = 0
    while n < len(buf):
        msg_len, new_pos = _DecodeVarint32(buf, n)
        n = new_pos
        msg_buf = buf[n:n+msg_len]
        try:
            msg = msgType()
            msg.ParseFromString(msg_buf)
            out.append(msg)
            n += msg_len
        except Exception as e:
            break
    return out

In [ ]:
# Compute Effective Sample Sizes for each algorithm
ESS = dict.fromkeys(algos)
for algo in algos:
    # Read chain
    chain = readManyFromFile(f"../{algo}.recordio", MarginalState)
    # Record number of clusters at each iteration
    n_clusters = np.empty(len(chain))
    for i in range(len(chain)):
        state = chain[i]
        n_clusters[i] = len(state.cluster_states)
    ESS[algo] = az.ess(n_clusters)

In [ ]:
# Times of MCMC, collected via the progressbar
filecoll_times = dict(zip(algos, [5.690, 6.824, 8.636]))
memocoll_times = dict(zip(algos, [5.617, 6.040, 7.348]))

In [ ]:
# Display computed ESS
for key, val in ESS.items():
    print(key, "ESS =", val, "-> ESS/time =", val/filecoll_times[key], sep="\t")

# Marginal (Neal2) vs conditional (BlockedGibbs)

Run Neal2:

In [ ]:
cmd = ("build/run algo_marg_settings.asciipb "
    "NNIG resources/asciipb/nnig_ngg_prior.asciipb "
    "DP   resources/asciipb/dp_gamma_prior.asciipb '' "
    "resources/csv/in/uni_data.csv resources/csv/in/uni_grid.csv "
    "resources/csv/out/uni_dens.csv resources/csv/out/uni_nclu.csv "
    "resources/csv/out/uni_clus.csv").split()
subprocess.run(cmd, capture_output=True)

Run Blocked Gibbs:

In [ ]:
cmd = ("../build/run ../algo_cond_settings.asciipb "
    "NNIG    ../resources/asciipb/nnig_ngg_prior.asciipb "
    "TruncSB ../resources/asciipb/truncsb_py_prior.asciipb '' "
    "../resources/csv/in/uni_data.csv ../resources/csv/in/uni_grid.csv "
    "../resources/csv/out/truncsb_dens.csv ../resources/csv/out/truncsb_nclu.csv "
    "../resources/csv/out/truncsb_clus.csv").split()
subprocess.run(cmd, capture_output=True)

Read density estimates:

In [ ]:
matr1 = np.genfromtxt("../resources/csv/out/uni_dens.csv", delimiter=',')
matr2 = np.genfromtxt("../resources/csv/out/truncsb_dens.csv", delimiter=',')

Plot density estimates:

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(uni_grid, np.exp(np.mean(matr1, axis=0)))
plt.plot(uni_grid, np.exp(np.mean(matr2, axis=0)))
plt.plot(uni_grid, true_pdf, color="red", linestyle="--")
plt.legend(["Neal2", "BlockedGibbs", "true"])
plt.title("Density estimates")